In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#install this library for face detection model
!pip install retina_face==0.0.13 --quiet

In [ ]:
#import the libraries
from tensorflow.keras.models import load_model
import numpy as np
import cv2
from retinaface import RetinaFace
# from google.colab.patches import cv2_imshow #colab patch for cv2
from tqdm import tqdm

In [ ]:
videopath="./01313031.mp4"
webcam = cv2.VideoCapture(videopath)
# webcam=cv2.VideoCapture(0) #for webcam

In [ ]:
frames=int(webcam.get(cv2.CAP_PROP_FRAME_COUNT))
frames

In [ ]:
gen_class = ['man','woman']
age_class = ['1-3','4-10','11-19','20-35','36-50','51-65','65+']

# load the gender and age pred models
model_age=load_model("./model/Age_prediction_model.h5")
model_gen=load_model("./model/Gender_prediction_model.h5")

#open video feed
webcam = cv2.VideoCapture(videopath)


frame_width = int(webcam.get(3))
frame_height = int(webcam.get(4))
fps=webcam.get(cv2.CAP_PROP_FPS)

#define parameters for saving the video after detection
fourcc = cv2.VideoWriter_fourcc(*'XVID')
new_my_video =new_my_video = videopath[:-4]+"_detect_gen_age3.avi"
out = cv2.VideoWriter(new_my_video, fourcc, 15, (frame_width, frame_height))

# loop through frames
count=0
while webcam.isOpened():
  status, frame = webcam.read()
  try:
    #detect the facial region with retinaface
    obj = RetinaFace.detect_faces(frame)
    try:
      for key in obj.keys():
        identify = obj[key]
        facial_area = identify["facial_area"]

        (startX, startY) = facial_area[2]+10, facial_area[3]+10
        (endX, endY) = facial_area[0]-10, facial_area[1]-10

        #draw bounding box around the face
        cv2.rectangle(frame, (startX,startY), (endX, endY), (0, 255, 0), 2)

        #copy the face region
        face_crop = np.copy(frame[endY:startY,endX:startX])

        face_crop = cv2.resize(face_crop, (224,224))
        face_crop = face_crop.astype("float")
        face_crop=face_crop.reshape(-1,224,224,3)


        #make predictions
        conf_gen = model_gen.predict(face_crop,verbose=0)[0]
        conf_age = model_age.predict(face_crop,verbose=0)[0]

        idx_gen = np.argmax(conf_gen)
        label_gen = gen_class[idx_gen]

        idx_age = np.argmax(conf_age)
        label_age = age_class[idx_age]

        #annotate on the frame
        label = "{}: {}".format(label_gen, label_age)

        Y = startY - 10 if startY - 10 > 10 else startY + 10

        cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 255, 0), 2)
    except:
      continue
    out.write(frame) #save the frame to the video
  except:
    continue

  count+=1
  print("{}/{} frames done".format(count,frames))
  if (count)==3000:
    break
  # cv2_imshow(frame)

  # press "Q" to stop
  if cv2.waitKey(1) & 0xFF == ord('q'):
      break

# release resources
webcam.release()
cv2.destroyAllWindows()